# Fine-tune the model for any of the benchmarks by editing the BENCHMAKR_NAME variable


In [1]:
import os
import random
import numpy as np
import pandas as pd
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

# Set random seeds for reproducibility
seed = 100
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Benchmark name and output specification
BENCHMARK_NAME = 'signalP_binary'
BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

# Directory containing benchmarks

# Loading the dataset
train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()

print(f'{len(train_set)} training set records, {len(valid_set)} validation set records, {len(test_set)} test set records.')
print(valid_set.head(10))

# Loading the pre-trained model and fine-tuning it on the loaded dataset
batch_size = 45
seq_len = 512
final = seq_len * 2
pretrained_model_generator, input_encoder = load_pretrained_model()

# get_model_with_hidden_layers_as_outputs gives the model output access to the hidden layers (on top of the output)
model_generator = FinetuningModelGenerator(pretrained_model_generator, OUTPUT_SPEC, pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs, dropout_rate=0.17681099042260753)

training_callbacks = [
    keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.25, min_lr=1e-05, verbose=1),
    keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
]

finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'],
         seq_len=seq_len, batch_size=batch_size, max_epochs_per_stage=1, lr=8.741510119145999e-05, begin_with_frozen_pretrained_layers=True,
         lr_with_frozen_pretrained_layers=1e-02, n_final_epochs=1, final_seq_len=final, final_lr=1e-05, callbacks=training_callbacks)

    # Evaluating the performance on the test set
results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['seq'], test_set['label'],
                                            start_seq_len=seq_len, start_batch_size=batch_size)
print("seq_len = %d" % seq_len)
print('Test-set performance:')
display(results)

print('Confusion matrix:')
display(confusion_matrix)

14945 training set records, 1661 validation set records, 4152 test set records.
       label                                                seq
13682      1  MEAPAQLLFLLLLWLPDTTGEIVMTQSPATLSVSPGERATLSCRAS...
14810      0  MAFISTPLGKVTVKSATVSANRRGLRMQSDSEPVVSRRALLSGALA...
5059       1  MARSKTAQPKHSLRKIAVVVATAVSGMSVYAQAAVEPKEDTITVTA...
6566       0  MSLLRIRKPKTRKGKKVLLAREPQLIESARTMLFLDGRKCGGNVKL...
10033      0  MWNSGFESYGSSSYGGAGGYTQSPGGFGSPAPSQAEKKSRARAQHI...
3300       0  MRKRISAIIMTLFMVLASCSNQLEAEKLAAESKNTFFDSLVKIGQG...
9804       0  MAPVVQQPAPSFKKTAVVDGVFEEVTLEQYKGKWVLLAFIPLAFTF...
3958       0  MAPKSDNTEAIVLNFVNEQNKPLNTQNAADALQKFNLKKTAVQKAL...
2713       0  MSYYQRPFSPSAYSLPASLNSSIVMQHGTSLDSTDTYPQHAQSLDG...
10760      0  MASSSNTMEFEEDDSTVTQTSLPTTTSVLDKSRSLFGNSAVISTPA...
[2024_07_23-15:47:14] Training set: Filtered out 0 of 14945 (0.0%) records of lengths exceeding 510.
[2024_07_23-15:47:15] Validation set: Filtered out 0 of 1661 (0.0%) records of lengths exceeding 510.
[2024_07_23-1

2024-07-23 15:47:15.400690: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3
2024-07-23 15:47:15.400714: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 8.00 GB
2024-07-23 15:47:15.400721: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 2.67 GB
2024-07-23 15:47:15.401289: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-07-23 15:47:15.401564: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2024-07-23 15:47:20.519097: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


575/575 [==============================] - ETA: 0s - loss: 0.4538WARNING:tensorflow:`evaluate()` received a value for `sample_weight`, but `weighted_metrics` were not provided.  Did you mean to pass metrics to `weighted_metrics` in `compile()`?  If this is intentional you can pass `weighted_metrics=[]` to `compile()` in order to silence this warning.


2024-07-23 15:48:54.412062: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


575/575 [==============================] - 110s 182ms/step - loss: 0.4538 - val_loss: 0.4501 - lr: 2.0000e-04
[2024_07_23-15:49:07] Training the entire fine-tuned model...


...

2024-07-23 16:03:03.614259: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


160/160 [==============================] - 32s 175ms/step
seq_len = 512
Test-set performance:


,# records,AUC
Model seq len,,
512,4152,0.501837
All,4152,0.501837


Confusion matrix:


,0,1
0,3478,0
1,674,0


# Finding number of uniquely labeled sequences

In [ ]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'
BENCHMARK_NAME = 'ProFET_NP_SP_Cleaved'
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0,1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
#train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()


print("Training Set pos : ", len(train_set[train_set["label"] == 1]))
print("Training Set neg : ", len(train_set[train_set["label"] == 0]))
print(len(train_set))
print("Testing Set pos : ", len(test_set[test_set["label"] == 1]))
print("Testing Set neg : ", len(test_set[test_set["label"] == 0]))
print(len(test_set))

In [ ]:
import os
import random
import numpy as np
import pandas as pd
from IPython.display import display
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'
BENCHMARK_NAME = 'scop'
OUTPUT_TYPE = OutputType(False, 'categorical')
UNIQUE_LABELS = ['a','b','c','d','e','f','g']
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)

train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=100)

test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()


print("Training Set a : ", len(train_set[train_set["label"] == 'a']))
print("Training Set b : ", len(train_set[train_set["label"] == 'b']))
print("Training Set c : ", len(train_set[train_set["label"] == 'c']))
print("Training Set d : ", len(train_set[train_set["label"] == 'd']))
print("Training Set e : ", len(train_set[train_set["label"] == 'e']))
print("Training Set f : ", len(train_set[train_set["label"] == 'f']))
print("Training Set g : ", len(train_set[train_set["label"] == 'g']))
print(len(train_set))
print("Valid Set a : ", len(valid_set[valid_set["label"] == 'a']))
print("Valid Set b  : ", len(valid_set[valid_set["label"] == 'b']))
print("Valid Set c : ", len(valid_set[valid_set["label"] == 'c']))
print("Valid Set d : ", len(valid_set[valid_set["label"] == 'd']))
print("Valid Set e : ", len(valid_set[valid_set["label"] == 'e']))
print("Valid Set f : ", len(valid_set[valid_set["label"] == 'f']))
print("Valid Set g : ", len(valid_set[valid_set["label"] == 'g']))
print(len(valid_set))
print("Testing Set a : ", len(test_set[test_set["label"] == 'a']))
print("Testing Set b : ", len(test_set[test_set["label"] == 'b']))
print("Testing Set c : ", len(test_set[test_set["label"] == 'c']))
print("Testing Set d : ", len(test_set[test_set["label"] == 'd']))
print("Testing Set e : ", len(test_set[test_set["label"] == 'e']))
print("Testing Set f : ", len(test_set[test_set["label"] == 'f']))
print("Testing Set g : ", len(test_set[test_set["label"] == 'g']))
print(len(test_set))

# Finding Embeddings

In [ ]:
from proteinbert import tokenization as tk
import pandas as pd
import csv
train_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/ProFET_NP_SP_Cleaved.train.csv"
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
f = open("/Users/yeyatiprasher/Coding/Internship/protein_bert/NPCleaved_embed.csv","w")
writer = csv.writer(f)
for i in train_set['seq']:
    embed = tk.tokenize_seq(i)
    writer.writerow(embed)

In [ ]:
from proteinbert import tokenization as tk
import pandas as pd
import csv
train_set_file_path = "/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks/scop.train.csv"
train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
f = open("/Users/yeyatiprasher/Coding/Internship/protein_bert/scop_embed.csv","w")
writer = csv.writer(f)
for i in train_set['seq']:
    embed = tk.tokenize_seq(i)
    writer.writerow(embed)

# Optimizing the model

In [ ]:
import os
import pandas as pd
import random
from IPython.display import display
import numpy as np
import tensorflow as tf

import optuna
from tensorflow import keras
from sklearn.model_selection import train_test_split

from proteinbert import OutputType, OutputSpec, FinetuningModelGenerator, load_pretrained_model, finetune, evaluate_by_len
from proteinbert.conv_and_global_attention_model import get_model_with_hidden_layers_as_outputs

BENCHMARK_NAME = 'ProFET_NP_SP_Cleaved'
BENCHMARKS_DIR = '/Users/yeyatiprasher/Coding/Internship/protein_bert/protein_benchmarks'

# A local (non-global) binary output
OUTPUT_TYPE = OutputType(False, 'binary')
UNIQUE_LABELS = [0, 1]
OUTPUT_SPEC = OutputSpec(OUTPUT_TYPE, UNIQUE_LABELS)




# Define the Optuna objective function
def objective(trial):
    # Suggest hyperparameters
    #batch_size = trial.suggest_int('batch_size', 20, 64)
    seed = trial.suggest_int('seed',80,110)
    #lr = trial.suggest_loguniform('lr', 1e-5, 1e-3)
    #dropout_rate = trial.suggest_uniform('dropout_rate', 0.1, 0.5)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)
    # Loading the dataset
    train_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.train.csv' % BENCHMARK_NAME)
    train_set = pd.read_csv(train_set_file_path).dropna().drop_duplicates()
    train_set, valid_set = train_test_split(train_set, stratify=train_set['label'], test_size=0.1, random_state=seed)

    test_set_file_path = os.path.join(BENCHMARKS_DIR, '%s.test.csv' % BENCHMARK_NAME)
    test_set = pd.read_csv(test_set_file_path).dropna().drop_duplicates()
    print('%d training set records, %d validation set records, %d test set records.' % (len(train_set), len(valid_set), len(test_set)))

    pretrained_model_generator, input_encoder = load_pretrained_model()
    model_generator = FinetuningModelGenerator(
        pretrained_model_generator, OUTPUT_SPEC, 
        pretraining_model_manipulation_function=get_model_with_hidden_layers_as_outputs,
        dropout_rate=0.17681099042260753
    )

    training_callbacks = [
        keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.25, min_lr=1e-05, verbose=1),
        keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
    ]

    finetune(model_generator, input_encoder, OUTPUT_SPEC, train_set['seq'], train_set['label'], valid_set['seq'], valid_set['label'], 
             seq_len=512, batch_size=32, max_epochs_per_stage=1, lr=8.741510119145999e-05, 
             begin_with_frozen_pretrained_layers=True, lr_with_frozen_pretrained_layers=1e-02, 
             n_final_epochs=1, final_seq_len=1024, final_lr=1e-05, callbacks=training_callbacks)

    results, confusion_matrix = evaluate_by_len(model_generator, input_encoder, OUTPUT_SPEC, test_set['seq'], test_set['label'], 
                                                start_seq_len=512, start_batch_size=32)
    
    overall_accuracy = results.loc['All', 'AUC']
    print(overall_accuracy)
    return overall_accuracy

# Create an Optuna study and optimize
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=5)

# Print the best hyperparameters
print('Best hyperparameters: ', study.best_params)
print('Best accuracy: ', study.best_value)